In [20]:
import pandas as pd

In [21]:
def clean_wiki(df):
    for c in df.columns:
        try:
            df[c] = df[c].str.replace('http://www.wikidata.org/entity/','')
        except:
            pass
    return df.fillna('')

In [22]:
raw_df = pd.read_csv('../data/entities/Q974.csv')
#before all else
raw_df = clean_wiki(raw_df)
[_ for _ in raw_df.columns if 'value' in _]

['business.value',
 'businessLabel.value',
 'origindate.value',
 'country.value',
 'industries.value',
 'ceos.value',
 'chairs.value',
 'hqs.value',
 'groups.value']

In [23]:
new_names = {'business.value':'id',
'businessLabel.value':'name',
'origindate.value':'start_date',
'country.value':'country_vals',
'industries.value':'industry_vals',
'ceos.value':'ceo_vals',
'chairs.value':'chair_vals',
'hqs.value':'hq_vals',
'groups.value':'group_vals',
'employees.value':'employee_count',#need to make bands for these
'profit.value':'profit',
'assets.value':'assets', 
'equity.value':'equity',
'markcap.value':'market_cap'}

In [27]:
df = pd.DataFrame(columns=list(new_names))
prev_cols = [_ for _ in list(new_names) if _ in raw_df.columns]
df[prev_cols] = raw_df[prev_cols]
df.columns = df.columns.map(new_names)
val_cols = [_ for _ in df.columns if '_vals' in _]
ent_df = df.drop(val_cols,axis=1)
ent_df = ent_df.fillna('').drop_duplicates()
ent_df

,id,name,start_date,employee_count,profit,assets,equity,market_cap
0,Q7370229,Rotana Hotels,1992-01-01T00:00:00Z,,,,,
4,Q84432349,Rainbow Sunrise Mapambazuko,2010-01-01T00:00:00Z,,,,,
9,Q21178738,Equity Bank Congo,2005-01-01T00:00:00Z,,,,,
12,Q65045748,Common Front for Congo,2018-01-01T00:00:00Z,,,,,
14,Q2996284,Convention of Christian Democrats,2005-08-13T00:00:00Z,,,,,
16,Q298507,Association Sportive Vita Club,1935-01-01T00:00:00Z,,,,,


In [28]:
mult_df = df[val_cols+['id']]
redex = mult_df.copy()
redex = redex.fillna('')
redex.index = mult_df['id']

In [29]:
def dic_to_ls(dic,typ):
    ret_ls = []
    for k, vs in dic.items():
        for v in vs:
            ret_ls.append({'company_id':k,'val_id':v,'typ':typ})
    return ret_ls

In [32]:
ind_ls = dic_to_ls(redex['industry_vals'].dropna().str.split(',').to_dict(), 'INDUSTRY')
ceo_ls = dic_to_ls(redex['ceo_vals'].dropna().str.split(',').to_dict(), 'CEO')
chair_ls = dic_to_ls(redex['chair_vals'].dropna().str.split(',').to_dict(), 'CHAIRMAN')
hq_ls = dic_to_ls(redex['hq_vals'].dropna().str.split(',').to_dict(), 'HEADQUARTERS')
group_ls = dic_to_ls(redex['group_vals'].dropna().str.split(',').to_dict(),'GROUPED_IN')
country_ls = dic_to_ls(redex['country_vals'].dropna().str.split(',').to_dict(),'RESIDES_IN')
ent_edge_df = pd.DataFrame(ind_ls+ceo_ls+chair_ls+hq_ls+group_ls+country_ls, columns = ['company_id','val_id','typ'])
ent_edge_df = ent_edge_df[ent_edge_df['val_id']!=''].drop_duplicates()

In [35]:
ent_edge_df

,company_id,val_id,typ
2,Q21178738,Q837171,INDUSTRY
18,Q7370229,Q1519,HEADQUARTERS
20,Q21178738,Q3838,HEADQUARTERS
21,Q65045748,Q3838,HEADQUARTERS
22,Q2996284,Q3838,HEADQUARTERS
23,Q298507,Q3838,HEADQUARTERS
30,Q7370229,Q796,RESIDES_IN
31,Q84432349,Q974,RESIDES_IN
32,Q21178738,Q974,RESIDES_IN
33,Q65045748,Q974,RESIDES_IN


In [48]:
raw_df = pd.read_csv('../data/edges/Q878.csv')
raw_df = clean_wiki(raw_df)
[_ for _ in raw_df.columns if 'value' in _]

['company.value', 'owner.value', 'acquiredate.value']

In [52]:
new_names = {'company.value':'owned_id',
'owner.value':'owner_id',
'acquiredate.value':'date'}

In [53]:
df = pd.DataFrame(columns=list(new_names))
prev_cols = [_ for _ in list(new_names) if _ in raw_df.columns]
df[prev_cols] = raw_df[prev_cols]
df.columns = df.columns.map(new_names)
edge_df = df.copy()
edge_df = edge_df.fillna('')

In [54]:
edge_df

,owned_id,owner_id,date
0,Q61971936,Q55613979,2020-07-01T00:00:00Z
1,Q460373,Q212304,
2,Q2544289,Q4115940,
3,Q4120269,Q543449,
